In [1]:
import numpy as np
import pandas as pd
import os
import csv
import zipfile
import re

In [2]:
src_dir = os.getcwd()
#if not os.path.exists(os.path.join(src_dir,"Oracle_Folders")):
 #   os.makedirs(os.path.join(src_dir,"Oracle_Folders"))
#dst_dir = os.path.join(src_dir,'Oracle_Folders')
dst_dir = os.path.join(src_dir,'ACTT Oracle Site')
dst_list_dir = os.listdir(dst_dir)
#outputs_dir = os.path.join(os.getcwd(),'Oracle_Outputs')
#output_zips = os.listdir(os.path.join(os.getcwd(),'Oracle_Outputs'))
vparameter_columns = ['DESCRIPTION VARCHAR2(255)', 'DISPLAY_VALUE VARCHAR2(4000)', 'HASH NUMBER(38,127)',
                      'ISADJUSTED VARCHAR2(5)', 'ISBASIC VARCHAR2(5)', 'ISDEFAULT VARCHAR2(9)',
                      'ISDEPRECATED VARCHAR2(5)', 'ISINSTANCE_MODIFIABLE VARCHAR2(5)', 'ISMODIFIED VARCHAR2(10)',
                      'ISSES_MODIFIABLE VARCHAR2(5)', 'ISSYS_MODIFIABLE VARCHAR2(9)', 'NAME VARCHAR2(80)',
                      'NUM NUMBER(38,127)', 'TYPE NUMBER(38,127)', 'UPDATE_COMMENT VARCHAR2(255)', 'VALUE VARCHAR2(4000)', 'DB']
vparameter = pd.DataFrame(columns=vparameter_columns)
dba_users_columns = ['ACCOUNT_STATUS VARCHAR2(32)', 'CREATED DATE(7)',
                    'DEFAULT_TABLESPACE VARCHAR2(30)', 'EDITIONS_ENABLED VARCHAR2(1)',
                    'EXPIRY_DATE DATE(7)', 'EXTERNAL_NAME VARCHAR2(4000)',
                    'INITIAL_RSRC_CONSUMER_GROUP VARCHAR2(128)', 'LOCK_DATE DATE(7)',
                    'PASSWORD VARCHAR2(4000)', 'PASSWORD_VERSIONS VARCHAR2(17)',
                    'PROFILE VARCHAR2(128)', 'TEMPORARY_TABLESPACE VARCHAR2(30)',
                    'USERNAME VARCHAR2(128)', 'USER_ID NUMBER(38,127)', 'DB']
dba_users = pd.DataFrame(columns=dba_users_columns)

dba_profiles_columns = ['LIMIT VARCHAR2(128)', 'PROFILE VARCHAR2(128)',
                        'RESOURCE_NAME VARCHAR2(32)', 'RESOURCE_TYPE VARCHAR2(8)','DB']
dba_profiles = pd.DataFrame(columns=dba_profiles_columns)

dba_defpass_columns = ['USERNAME VARCHAR2(128)', 'DB']
dba_defpass = pd.DataFrame(columns=dba_defpass_columns)

dba_sys_privs_columns = ['ADMIN_OPTION VARCHAR2(3)', 'GRANTEE VARCHAR2(128)', 'PRIVILEGE VARCHAR2(40)', 'DB']
dba_sys_privs = pd.DataFrame(columns=dba_sys_privs_columns)

dba_tab_privs_columns = ['GRANTABLE VARCHAR2(3)', 'GRANTEE VARCHAR2(128)', 'GRANTOR VARCHAR2(128)',
                         'HIERARCHY VARCHAR2(3)', 'OWNER VARCHAR2(128)', 'PRIVILEGE VARCHAR2(40)',
                         'TABLE_NAME VARCHAR2(128)', 'DB']
dba_tab_privs = pd.DataFrame(columns=dba_tab_privs_columns)

dba_role_privs_columns = ['ADMIN_OPTION VARCHAR2(3)', 'DEFAULT_ROLE VARCHAR2(3)', 'GRANTED_ROLE VARCHAR2(128)',
                          'GRANTEE VARCHAR2(128)']
dba_role_privs = pd.DataFrame(columns=dba_role_privs_columns)
oracle_tc = pd.read_csv(os.path.join(os.getcwd(),'Oracle_TC.csv'), sep=";")
nomina = pd.read_excel('nomina.xlsx')
nomina['Nombre de usuario'] =  nomina['Nombre de usuario'].apply(lambda x: str(x).upper())
nomina_externos = pd.read_excel('nomina_externos.xlsx')
nomina_externos['NOMBRE_USUARIO'] = nomina_externos['NOMBRE_USUARIO'].apply(lambda x: str(x).upper()).apply(lambda x: x.replace(" ",""))
nomina_externos.rename(columns={'DEPARTAMENTO':'Departamento_ext'}, inplace=True)
critical_bd = pd.read_excel('Universo_SOX_Oracle.xlsx')
critical_bd['SCHEMA'] = critical_bd['SCHEMA'].apply(lambda x: str(x).upper()).apply(lambda x: x.replace(" ",""))

In [3]:
dst_list_dir

['bilw01', 'crtw01', 'loyw01', 'payw01', 'priw01']

In [4]:
dst_dir

'C:\\Users\\tlopezradits\\OneDrive - Deloitte (O365D)\\MELI 2021 Primer Semestre\\Oracle Scripts\\new\\ACTT Oracle Site'

In [5]:
for o in dst_list_dir:
    
    df_vparameter = pd.read_csv(os.path.join(dst_dir,o,'VPARAMETER.ACTT'), sep='[|^|]', engine='python')
    df_vparameter = df_vparameter.loc[:, ~df_vparameter.columns.str.contains('^Unnamed')]
    df_vparameter['DB'] = re.search(r'[a-zA-Z0-9-_]+', o).group().upper()
    vparameter = pd.concat([vparameter, df_vparameter], names=vparameter_columns, sort=False)
    
    users = pd.read_csv(os.path.join(dst_dir,o,'DBA_USERS.ACTT'), sep='[|^|]', engine='python')
    users = users.loc[:, ~users.columns.str.contains('^Unnamed')]
    users['DB'] = re.search(r'[a-zA-Z0-9-_]+', o).group().upper()
    dba_users = pd.concat([dba_users, users], names=dba_users_columns, sort=False)
    
    profiles = pd.read_csv(os.path.join(dst_dir,o,'DBA_PROFILES.ACTT'), sep='[|^|]', engine='python')
    profiles = profiles.loc[:, ~profiles.columns.str.contains('^Unnamed')]
    profiles['DB'] = re.search(r'[a-zA-Z0-9-_]+', o).group().upper()
    dba_profiles = pd.concat([dba_profiles, profiles], names=dba_profiles_columns, sort=False)    
    
    df_defpass = pd.read_csv(os.path.join(dst_dir,o,'DBA_USERS_WITH_DEFPWD.ACTT'), sep='[|^|]', engine='python')
    df_defpass = df_defpass.loc[:, ~df_defpass.columns.str.contains('^Unnamed')]
    df_defpass['DB'] = re.search(r'[a-zA-Z0-9-_]+', o).group().upper()
    dba_defpass = pd.concat([dba_defpass, df_defpass], names=dba_defpass_columns, sort=False)
    
    df_sys_privs = pd.read_csv(os.path.join(dst_dir,o,'DBA_SYS_PRIVS.ACTT'), sep='[|^|]', engine='python')
    df_sys_privs = df_sys_privs.loc[:, ~df_sys_privs.columns.str.contains('^Unnamed')]
    df_sys_privs['DB'] = re.search(r'[a-zA-Z0-9-_]+', o).group().upper()
    dba_sys_privs = pd.concat([dba_sys_privs, df_sys_privs], names=dba_sys_privs_columns, sort=False)
    
    df_tab_privs = pd.read_csv(os.path.join(dst_dir,o,'DBA_TAB_PRIVS.ACTT'), sep='[|^|]', engine='python')
    df_tab_privs = df_tab_privs.loc[:, ~df_tab_privs.columns.str.contains('^Unnamed')]
    df_tab_privs['DB'] = re.search(r'[a-zA-Z0-9-_]+', o).group().upper()
    dba_tab_privs = pd.concat([dba_tab_privs, df_tab_privs], names=dba_tab_privs_columns, sort=False)
    
    df_role_privs = pd.read_csv(os.path.join(dst_dir,o,'DBA_ROLE_PRIVS.ACTT'), sep='[|^|]', engine='python')
    df_role_privs = df_role_privs.loc[:, ~df_role_privs.columns.str.contains('^Unnamed')]
    df_role_privs['DB'] = re.search(r'[a-zA-Z0-9-_]+', o).group().upper()
    dba_role_privs = pd.concat([dba_role_privs, df_role_privs], names=dba_role_privs_columns, sort=False)

In [6]:
vparameter = vparameter[['NAME VARCHAR2(80)', 'VALUE VARCHAR2(4000)', 'DESCRIPTION VARCHAR2(255)','DB']].applymap(lambda x: str(x).strip())
vparameter[['NAME VARCHAR2(80)', 'VALUE VARCHAR2(4000)','DB']] = vparameter[['NAME VARCHAR2(80)', 'VALUE VARCHAR2(4000)','DB']].applymap(lambda x: str(x).replace("  ",""))
vparameter.rename(columns ={'NAME VARCHAR2(80)':'NAME',
                            'VALUE VARCHAR2(4000)':'VALUE',
                            'DESCRIPTION VARCHAR2(255)': 'DESCRIPTION'}, inplace=True)

dba_users = dba_users[['USERNAME VARCHAR2(128)', 'ACCOUNT_STATUS VARCHAR2(32)', 'PROFILE VARCHAR2(128)','EXTERNAL_NAME VARCHAR2(4000)','DB']].applymap(lambda x: str(x).strip()).applymap(lambda x: str(x).replace("  ",""))
dba_users.rename(columns ={'USERNAME VARCHAR2(128)':'USERNAME',
                           'ACCOUNT_STATUS VARCHAR2(32)':'ACCOUNT_STATUS',
                           'PROFILE VARCHAR2(128)':'PROFILE',
                           'EXTERNAL_NAME VARCHAR2(4000)':'EXTERNAL_NAME'}, inplace=True)
dba_profiles.rename(columns={'LIMIT VARCHAR2(128)': 'LIMIT', 
                             'PROFILE VARCHAR2(128)': 'PROFILE',
                             'RESOURCE_NAME VARCHAR2(32)': 'RESOURCE_NAME',
                             'RESOURCE_TYPE VARCHAR2(8)': 'RESOURCE_TYPE'}, inplace=True)
dba_sys_privs = dba_sys_privs.applymap(lambda x: str(x).strip()).applymap(lambda x: str(x).replace("  ",""))
dba_sys_privs.rename(columns={'ADMIN_OPTION VARCHAR2(3)':'ADMIN_OPTION ',
                             'GRANTEE VARCHAR2(128)':'GRANTEE',
                             'PRIVILEGE VARCHAR2(40)':'PRIVILEGE'}, inplace=True)
dba_tab_privs = dba_tab_privs[['GRANTEE VARCHAR2(128)', 'OWNER VARCHAR2(128)',
                             'PRIVILEGE VARCHAR2(40)', 'TABLE_NAME VARCHAR2(128)', 'DB']].applymap(lambda x: str(x).strip()).applymap(lambda x: str(x).replace("  ",""))
dba_tab_privs.rename(columns={'GRANTEE VARCHAR2(128)':'GRANTEE', 
                              'OWNER VARCHAR2(128)':'OWNER',
                              'PRIVILEGE VARCHAR2(40)':'PRIVILEGE', 
                              'TABLE_NAME VARCHAR2(128)':'TABLE_NAME'}, inplace=True)
dba_role_privs = dba_role_privs.applymap(lambda x: str(x).strip()).applymap(lambda x: str(x).replace("  ",""))
dba_role_privs.rename(columns={'ADMIN_OPTION VARCHAR2(3)':'ADMIN_OPTION',
                               'DEFAULT_ROLE VARCHAR2(3)':'DEFAULT_ROLE',
                               'GRANTED_ROLE VARCHAR2(128)':'GRANTED_ROLE',
                               'GRANTEE VARCHAR2(128)':'GRANTEE'}, inplace=True)

In [7]:
vparameter_mask = (vparameter['NAME'] == 'os_authent_prefix') | \
                    (vparameter['NAME'] == 'os_roles') | \
                    (vparameter['NAME'] == 'remote_os_roles')
vparameter = vparameter[vparameter_mask].reset_index(drop=True)

mask_dba_users = (dba_users['ACCOUNT_STATUS'] == 'OPEN') | \
                    (dba_users['ACCOUNT_STATUS'] == 'EXPIRED') | \
                    (dba_users['ACCOUNT_STATUS'] == 'EXPIRED(GRACE)')
dba_users = dba_users[mask_dba_users].reset_index(drop=True)

In [8]:
sys_privs = ['ALTER DATABASE', 'ALTER ANY PROCEDURE', 'ALTER ANY ROLE', 'ALTER ANY TABLE', 
            'ALTER PROFILE', 'ALTER SYSTEM', 'ALTER USER', 'BECOME USER', 'CREATE ANY CLUSTER', 
            'CREATE ANY TABLE', 'CREATE PROFILE', 'CREATE ROLE', 'CREATE USER', 'DELETE ANY TABLE',
            'DROP ANY LIBRARY', 'DROP ANY PROCEDURE', 'DROP ANY ROLE', 'DROP ANY TABLE',
            'DROP PROFILE', 'DROP TABLESPACE', 'DROP USER', 'GRANT ANY PRIVILEGE', 'GRANT ANY ROLE',
            'INSERT ANY TABLE', 'RESTRICTED SESSION', 'UPDATE ANY TABLE']

dba_sys_privs_critical = pd.DataFrame(columns=list(dba_sys_privs.columns))
for priv in sys_privs:
    sys_privs_mask = (dba_sys_privs['PRIVILEGE'] == priv)
    dba_sys_privs_critical = pd.concat([dba_sys_privs_critical, dba_sys_privs[sys_privs_mask]], sort=False).reset_index(drop=True)

del dba_sys_privs

In [9]:
productive_schemas = list(critical_bd['SCHEMA'].unique())
sys_schemas = list(['SYS'])
critical_schemas = sys_schemas + productive_schemas

dba_tab_privs_schemas = pd.DataFrame(columns=list(dba_tab_privs.columns))
for schema in critical_schemas:
    tab_schema_mask = (dba_tab_privs['OWNER'] == schema)
    dba_tab_privs_schemas = pd.concat([dba_tab_privs_schemas, dba_tab_privs[tab_schema_mask]], sort=False).reset_index(drop=True)


In [10]:
tab_privs = ['INSERT', 'UPDATE', 'DELETE', 'ALTER']
dba_tab_privs_critical = pd.DataFrame(columns=list(dba_tab_privs.columns))
for priv in tab_privs:
    tab_privs_mask = (dba_tab_privs_schemas['PRIVILEGE'] == priv)
    dba_tab_privs_critical = pd.concat([dba_tab_privs_critical, dba_tab_privs_schemas[tab_privs_mask]], sort=False).reset_index(drop=True)
    
del dba_tab_privs
del dba_tab_privs_schemas

In [11]:
dba_role_sysprivs_critical = pd.merge(dba_role_privs, dba_sys_privs_critical, how='inner', left_on=['GRANTED_ROLE','DB'],
                                      right_on=['GRANTEE','DB'], validate='many_to_many').drop_duplicates().reset_index(drop=True)
dba_role_tabprivs_critical = pd.merge(dba_role_privs, dba_tab_privs_critical, how='inner', left_on=['GRANTED_ROLE','DB'],
                                      right_on=['GRANTEE','DB'], validate='many_to_many').drop_duplicates().reset_index(drop=True)


In [12]:
dba_tab_privs_critical = pd.merge(dba_tab_privs_critical, oracle_tc[['schema_name','db_name','table_name']],
                                  how='left', left_on='TABLE_NAME', right_on='table_name', 
                                  validate='many_to_many').drop_duplicates().reset_index(drop=True)
dba_tab_privs_critical.rename(columns={'table_name':'TABLA_CRITICA',
                                      'schema_name': 'CRITICAL_SCHEMA_NAME',
                                      'db_name': 'CRITICAL_DB_NAME'}, inplace=True)

In [13]:
dba_role_tabprivs_critical = pd.merge(dba_role_tabprivs_critical, oracle_tc[['schema_name','db_name','table_name']],
                                  how='left', left_on='TABLE_NAME', right_on='table_name', 
                                  validate='many_to_many').drop_duplicates().reset_index(drop=True)
dba_role_tabprivs_critical.rename(columns={'table_name':'TABLA_CRITICA',
                                      'schema_name': 'CRITICAL_SCHEMA_NAME',
                                      'db_name': 'CRITICAL_DB_NAME'}, inplace=True)

In [14]:
manual_tc = ['(^DBA_USERS$)', '(^USER$)', '(^DBA_SYS_PRIVS$)', '(^DBA_TAB_PRIVS$)', '(^DBA_ROLE_PRIVS$)', '(^DBA_PROFILES$)', '(^DBA_OBJECTS$)']
c1 = dba_tab_privs_critical['TABLE_NAME'].str.match('|'.join(manual_tc))
dba_tab_privs_critical['TABLA_CRITICA'][dba_tab_privs_critical['TABLA_CRITICA'].isnull()] = np.select([c1], [dba_tab_privs_critical['TABLA_CRITICA'].fillna(1)])
dba_tab_privs_critical['TABLA_CRITICA'].loc[dba_tab_privs_critical['TABLA_CRITICA'] != 0] = 1
c2 = dba_role_tabprivs_critical['TABLE_NAME'].str.match('|'.join(manual_tc))
dba_role_tabprivs_critical['TABLA_CRITICA'][dba_role_tabprivs_critical['TABLA_CRITICA'].isnull()] = np.select([c2], [dba_role_tabprivs_critical['TABLA_CRITICA'].fillna(1)])
dba_role_tabprivs_critical['TABLA_CRITICA'].loc[dba_role_tabprivs_critical['TABLA_CRITICA'] != 0] = 1

In [15]:
dba_tab_privs_critical = pd.merge(dba_tab_privs_critical, nomina[['Nombre de usuario', 'Departamento']], how='left', left_on='GRANTEE', right_on='Nombre de usuario').drop('Nombre de usuario', axis=1)
dba_tab_privs_critical = pd.merge(dba_tab_privs_critical, nomina_externos[['NOMBRE_USUARIO', 'Departamento_ext']], how='left', left_on='GRANTEE', right_on='NOMBRE_USUARIO').drop('NOMBRE_USUARIO', axis=1)
dba_tab_privs_critical['Comentario'] = 'N/A'

In [16]:
dba_role_tabprivs_critical = pd.merge(dba_role_tabprivs_critical, nomina[['Nombre de usuario', 'Departamento']], how='left', left_on='GRANTEE_x', right_on='Nombre de usuario').drop('Nombre de usuario', axis=1)
dba_role_tabprivs_critical = pd.merge(dba_role_tabprivs_critical, nomina_externos[['NOMBRE_USUARIO', 'Departamento_ext']], how='left', left_on='GRANTEE_x', right_on='NOMBRE_USUARIO').drop('NOMBRE_USUARIO', axis=1)
dba_role_tabprivs_critical['Comentario'] = 'N/A'

In [17]:
comment_mask = (dba_tab_privs_critical['Departamento'].isnull()) & (dba_tab_privs_critical['Departamento_ext'].isnull()) & (dba_tab_privs_critical['TABLA_CRITICA'] == 0)

In [18]:
dba_tab_privs_critical['Comentario'][comment_mask] = 'Privilegio no relevante, tabla no crítica'

In [19]:
comment_mask = (dba_role_tabprivs_critical['Departamento'].isnull()) & (dba_role_tabprivs_critical['Departamento_ext'].isnull()) & (dba_role_tabprivs_critical['TABLA_CRITICA'] == 0)


In [20]:
dba_role_tabprivs_critical['Comentario'][comment_mask] = 'Privilegio no relevante, tabla no crítica'

In [21]:
common_user_mask = (dba_tab_privs_critical['Departamento'].isnull()) & (dba_tab_privs_critical['Departamento_ext'].isnull()) & (dba_tab_privs_critical['Comentario'] == 'N/A')
common_user_mask2 = (dba_role_tabprivs_critical['Departamento'].isnull()) & (dba_role_tabprivs_critical['Departamento_ext'].isnull()) & (dba_role_tabprivs_critical['Comentario'] == 'N/A')

In [22]:
common_users = ['SYS','MIGRA', 'SEGINF', 'APP_ORACLE_GANDALF', 'GGAAS', 'GGS',
                'GSM','APEX_', 'MONITOR', 'SPATIAL_CSW_ADMIN_USR',
                'ALTER_QA', 'BULK_DB', 'CONSULTAS_EXC', '_READONLY']
own_default_users = ('_W01', 'W01PROD', 'W01PROD2', 'W02PROD', '_W_I1', '_W_I2', 'DBPROD')
c1 = dba_tab_privs_critical['GRANTEE'].str.contains('|'.join(common_users))
c2 = dba_tab_privs_critical['GRANTEE'].str.endswith(own_default_users)
c3 = dba_role_tabprivs_critical['GRANTEE_x'].str.contains('|'.join(common_users))
c4 = dba_role_tabprivs_critical['GRANTEE_x'].str.endswith(own_default_users)

dba_tab_privs_critical['Comentario'][common_user_mask] = np.select([c1,c2], 
                                                                          [dba_tab_privs_critical['Comentario'].replace('N/A','Usuario de gestion de aplicativo comun a todas las bases'),
                                                                           dba_tab_privs_critical['Comentario'].replace('N/A','Usuario de schema comun a todas las bases al momento de crearlas')])
dba_role_tabprivs_critical['Comentario'][common_user_mask2] = np.select([c3,c4], 
                                                                          [dba_role_tabprivs_critical['Comentario'].replace('N/A','Usuario de gestion de aplicativo comun a todas las bases'),
                                                                           dba_role_tabprivs_critical['Comentario'].replace('N/A','Usuario de schema comun a todas las bases al momento de crearlas')])

In [23]:
dba_sys_privs_critical.columns

Index(['ADMIN_OPTION ', 'GRANTEE', 'PRIVILEGE', 'DB'], dtype='object')

In [24]:
dba_role_sysprivs_critical.columns

Index(['ADMIN_OPTION', 'DEFAULT_ROLE', 'GRANTED_ROLE', 'GRANTEE_x', 'DB',
       'ADMIN_OPTION ', 'GRANTEE_y', 'PRIVILEGE'],
      dtype='object')

In [25]:
critical_roles = pd.DataFrame(dba_role_sysprivs_critical['GRANTED_ROLE'].drop_duplicates().reset_index(drop=True))
critical_roles.rename(columns={'GRANTED_ROLE':'CRITICAL_ROLE'}, inplace=True)

In [26]:
critical_roles.columns

Index(['CRITICAL_ROLE'], dtype='object')

In [27]:
critical_roles

,CRITICAL_ROLE
0,DBA
1,EM_EXPRESS_ALL
2,GSMADMIN_ROLE
3,DATAPUMP_IMP_FULL_DATABASE
4,IMP_FULL_DATABASE


In [28]:
dba_sys_privs_critical = pd.merge(dba_sys_privs_critical, nomina[['Nombre de usuario', 'Departamento']], how='left', left_on='GRANTEE', right_on='Nombre de usuario').drop('Nombre de usuario', axis=1)
dba_sys_privs_critical = pd.merge(dba_sys_privs_critical, nomina_externos[['NOMBRE_USUARIO', 'Departamento_ext']], how='left', left_on='GRANTEE', right_on='NOMBRE_USUARIO').drop('NOMBRE_USUARIO', axis=1)
dba_sys_privs_critical = pd.merge(dba_sys_privs_critical, critical_roles, how='left', left_on='GRANTEE', right_on='CRITICAL_ROLE')
dba_sys_privs_critical['CRITICAL_ROLE'][dba_sys_privs_critical['CRITICAL_ROLE'].notnull()] = 1
dba_sys_privs_critical['CRITICAL_ROLE'][dba_sys_privs_critical['CRITICAL_ROLE'].isnull()] = 0
common_user_mask = (dba_sys_privs_critical['Departamento'].isnull()) & (dba_sys_privs_critical['Departamento_ext'].isnull()) & (dba_sys_privs_critical['CRITICAL_ROLE'] == 0)
dba_sys_privs_critical['Comentario'] = np.NaN
c1 = dba_sys_privs_critical['GRANTEE'].str.contains('|'.join(common_users))
c2 = dba_sys_privs_critical['GRANTEE'].str.endswith(own_default_users)
dba_sys_privs_critical['Comentario'][common_user_mask] = np.select([c1,c2], 
                                                                          [dba_sys_privs_critical['Comentario'].fillna('Usuario de gestion de aplicativo comun a todas las bases'),
                                                                           dba_sys_privs_critical['Comentario'].fillna('Usuario de schema comun a todas las bases al momento de crearlas')])
dba_sys_privs_critical['Comentario'][dba_sys_privs_critical['CRITICAL_ROLE'] == 1] = 'Se corresponde con un rol'

<ipython-input-28-e4c0bc4d5663>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dba_sys_privs_critical['Comentario'][common_user_mask] = np.select([c1,c2],


In [29]:
dba_sys_privs_critical.head()

,ADMIN_OPTION,GRANTEE,PRIVILEGE,DB,Departamento,Departamento_ext,CRITICAL_ROLE,Comentario
0,NO,IMP_FULL_DATABASE,ALTER DATABASE,BILW01,NaN,NaN,1,Se corresponde con un rol
1,NO,SYSBACKUP,ALTER DATABASE,BILW01,NaN,NaN,0,Usuario de gestion de aplicativo comun a todas...
2,NO,DBA,ALTER DATABASE,BILW01,NaN,NaN,1,Se corresponde con un rol
3,NO,SYS,ALTER DATABASE,BILW01,NaN,NaN,0,Usuario de gestion de aplicativo comun a todas...
4,NO,DATAPUMP_IMP_FULL_DATABASE,ALTER DATABASE,BILW01,NaN,NaN,1,Se corresponde con un rol


In [30]:
granted_roles = list(dba_role_sysprivs_critical['GRANTED_ROLE'].unique())
dba_role_sysprivs_critical = pd.merge(dba_role_sysprivs_critical, nomina[['Nombre de usuario', 'Departamento']], how='left', left_on='GRANTEE_x', right_on='Nombre de usuario').drop('Nombre de usuario', axis=1)
dba_role_sysprivs_critical = pd.merge(dba_role_sysprivs_critical, nomina_externos[['NOMBRE_USUARIO', 'Departamento_ext']], how='left', left_on='GRANTEE_x', right_on='NOMBRE_USUARIO').drop('NOMBRE_USUARIO', axis=1)
c1 = dba_role_sysprivs_critical['GRANTEE_x'].str.contains('|'.join(common_users))
c2 = dba_role_sysprivs_critical['GRANTEE_x'].str.endswith(own_default_users)
c3 = dba_role_sysprivs_critical['GRANTEE_x'].str.contains('|'.join(granted_roles))
common_user_mask = (dba_role_sysprivs_critical['Departamento'].isnull()) & (dba_role_sysprivs_critical['Departamento_ext'].isnull())
dba_role_sysprivs_critical['Comentario'] = np.NaN
dba_role_sysprivs_critical['Comentario'][common_user_mask] = np.select([c1,c2,c3], 
                                                                          [dba_role_sysprivs_critical['Comentario'].fillna('Usuario de gestion de aplicativo comun a todas las bases'),
                                                                           dba_role_sysprivs_critical['Comentario'].fillna('Usuario de schema comun a todas las bases al momento de crearlas'),
                                                                           dba_role_sysprivs_critical['Comentario'].fillna('Corresponde a un rol dentro de otro rol')])

<ipython-input-30-7f3ebef19080>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dba_role_sysprivs_critical['Comentario'][common_user_mask] = np.select([c1,c2,c3],


In [31]:
critical_sys_users = dba_sys_privs_critical[['GRANTEE', 'DB']].drop_duplicates().reset_index(drop=True)
critical_sys_users_byrole = dba_role_sysprivs_critical[['GRANTEE_x', 'DB']].drop_duplicates().reset_index(drop=True)
critical_sys_users_byrole.rename(columns={'GRANTEE_x':'GRANTEE'}, inplace=True)
critical_tab_users = dba_tab_privs_critical[['GRANTEE', 'DB','CRITICAL_SCHEMA_NAME']].drop_duplicates().reset_index(drop=True)
critical_tab_users = critical_tab_users[critical_tab_users['CRITICAL_SCHEMA_NAME'].notnull()].drop_duplicates().reset_index(drop=True)
critical_tab_users = critical_tab_users[['GRANTEE', 'DB']].drop_duplicates().reset_index(drop=True)
critical_tab_users_byrole = dba_role_tabprivs_critical[['GRANTEE_x', 'DB']].drop_duplicates().reset_index(drop=True)
critical_tab_users_byrole.rename(columns={'GRANTEE_x':'GRANTEE'}, inplace=True)
total_critical_users = pd.concat([critical_sys_users,critical_sys_users_byrole,critical_tab_users,critical_tab_users_byrole], axis=0).drop_duplicates().reset_index(drop=True)
total_critical_users.rename(columns={'GRANTEE':'USERNAME'}, inplace=True)
critical_profiles = pd.merge(dba_users[['USERNAME','PROFILE', 'EXTERNAL_NAME','DB']], total_critical_users, how='left', on=['USERNAME','DB'])
critical_profiles = critical_profiles[critical_profiles['PROFILE'].notnull()]
critical_profiles.rename(columns= {'PROFILE':'CRITICAL_PROFILE'}, inplace=True)

In [32]:
dba_profiles = pd.merge(dba_profiles,critical_profiles[['CRITICAL_PROFILE','DB']].drop_duplicates().reset_index(drop=True), how='left', left_on=['PROFILE','DB'], right_on=['CRITICAL_PROFILE','DB'])
dba_profiles['CRITICAL_PROFILE'][dba_profiles['CRITICAL_PROFILE'].notnull()] = 1
dba_profiles['CRITICAL_PROFILE'][dba_profiles['CRITICAL_PROFILE'].isnull()] = 0
dba_profiles['Comentario'] = np.nan
dba_profiles['Comentario'][dba_profiles['CRITICAL_PROFILE'] == 0] = 'Valores no relevantes, perfil sin usuarios críticos'

<ipython-input-32-21a01502cb22>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dba_profiles['Comentario'][dba_profiles['CRITICAL_PROFILE'] == 0] = 'Valores no relevantes, perfil sin usuarios críticos'


In [33]:
dba_sys_privs_critical.rename(columns={'GRANTEE':'USER/ROLE'}, inplace=True)
dba_role_sysprivs_critical.rename(columns={'GRANTEE_x':'USER/ROLE'}, inplace=True)
dba_tab_privs_critical.rename(columns={'GRANTEE':'USER/ROLE'}, inplace=True)
dba_role_tabprivs_critical.rename(columns={'GRANTEE_x':'USER/ROLE'}, inplace=True)

In [34]:
dba_tab_privs_critical = dba_tab_privs_critical.fillna(value="-")
dba_role_tabprivs_critical = dba_role_tabprivs_critical.fillna(value="-")
dba_sys_privs_critical = dba_sys_privs_critical.fillna(value="-")
dba_role_sysprivs_critical = dba_role_sysprivs_critical.fillna(value="-")
dba_users = dba_users.fillna(value="-")
dba_profiles = dba_profiles.fillna(value="-")

In [36]:
dba_tab_privs_critical.to_csv('DBA_TAB_PRIVS_FINAL.csv')
dba_role_tabprivs_critical.to_csv('DBA_ROLE_TABPRIVS_FINAL.csv')
dba_sys_privs_critical.to_csv('DBA_SYS_PRIVS_FINAL.csv')
dba_role_sysprivs_critical.to_csv('DBA_ROLE_SYSPRIVS_FINAL.csv')
dba_users.to_csv('DBA_USERS_FINAL.csv')
dba_profiles.to_csv('DBA_PROFILES_FINAL.csv')
critical_profiles.to_csv('CRITICAL_USERS_PROFILES.csv')
vparameter.to_csv('VPARAMETER_FINAL.csv')